# MED-PC Extracting the Recording Data and Metadata

## Importing the Python Libraries

In [1]:
import sys
import glob
from collections import defaultdict
import os
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from moviepy.editor import *
from datetime import datetime
from IPython.display import Video

In [2]:
from medpc2excel.medpc_read import medpc_read

In [3]:
# setting path
sys.path.append('../../src')

In [4]:
# All the libraries that were created for this repository
import extract.dataframe
import processing.tone
import extract.metadata

## Getting the Metadata from all the MED-PC Recording Files

- Getting all the file paths of the recording files(that happen to all end in `.txt`

# NOTE: The following path must be changed to the directory where your MED-PC recording files are located, if they are not in the specied folder

In [5]:
all_med_pc_file = sorted(glob.glob("./data/**/*.txt"))

- Use this instead if you're using you're own data

In [6]:
all_med_pc_file[:10]

['./data/med_pc_recording_files/2022-10-03_09h45m_Subject 1.1v1.2.txt',
 './data/med_pc_recording_files/2022-10-03_09h45m_Subject 1.3v1.4.txt',
 './data/med_pc_recording_files/2022-10-03_09h45m_Subject 2.1v2.2.txt',
 './data/med_pc_recording_files/2022-10-03_09h45m_Subject 2.3v2.4.txt',
 './data/med_pc_recording_files/2022-10-03_10h57m_Subject 3.1v3.2.txt',
 './data/med_pc_recording_files/2022-10-03_10h57m_Subject 3.3v3.4.txt',
 './data/med_pc_recording_files/2022-10-03_10h57m_Subject 4.1v4.2.txt',
 './data/med_pc_recording_files/2022-10-03_10h57m_Subject 4.3v4.4.txt',
 './data/med_pc_recording_files/2022-10-03_11h07m_Subject 3.1v3.2.txt',
 './data/med_pc_recording_files/2022-10-03_11h07m_Subject 3.3v3.4.txt']

- Selecting only one of the files

In [7]:
all_med_pc_file = [all_med_pc_file[0]]

- Example of what the MED-PC Recording file looks like

In [8]:
with open(all_med_pc_file[0]) as f:
    lines = f.readlines()
    for line in lines[:20]:
        print(line)

File: C:\MED-PC\Data\2022-10-03_09h45m_Subject 1.1v1.2.txt







Start Date: 10/03/22

End Date: 10/03/22

Subject: 1.1v1.2

Experiment: CD1_vs_C57_Comparison

Group: Cage_1

Box: 1

Start Time: 09:45:25

End Time: 10:46:28

MSN: C57_reward_competition

A:    5399.000

D:    8000.000

E:       0.000

L:       0.000

M:       0.000

O:       0.000

T:    1860.000



- We will be extracting the first 10 or so lines that look like:

```
File: C:\MED-PC\Data\2022-05-06_12h59m_Subject 3.4 (2).txt

Start Date: 05/06/22

End Date: 05/06/22

Subject: 3.4 (2)

Experiment: Pilot of Pilot

Group: Cage 4

Box: 1

Start Time: 12:59:58

End Time: 14:02:38

MSN: levelNP_CS_reward_laserepochON1st_noshock
```
    
- We will just find all the lines that start with "File", "Start Date", "End Date", "Subject", "Experiment", "Group", "Box", "Start Time", "End Time", or "MSN". And then stop once all the metadata types have been collected

In [9]:
# This makes a nested dictionary of file paths to each individual metadata type
file_path_to_meta_data = extract.metadata.get_all_med_pc_meta_data_from_files(list_of_files=all_med_pc_file)

In [10]:
for key, value in file_path_to_meta_data.items():
    print("File path: {}".format(key))
    print("Metadata types and associated values: {}".format(value))
    break

File path: ./data/med_pc_recording_files/2022-10-03_09h45m_Subject 1.1v1.2.txt
Metadata types and associated values: {'File': 'C:\\MED-PC\\Data\\2022-10-03_09h45m_Subject 1.1v1.2.txt', 'Start Date': '10/03/22', 'End Date': '10/03/22', 'Subject': '1.1v1.2', 'Experiment': 'CD1_vs_C57_Comparison', 'Group': 'Cage_1', 'Box': '1', 'Start Time': '09:45:25', 'End Time': '10:46:28', 'MSN': 'C57_reward_competition'}


## Making a Dataframe out of the Metadata

In [11]:
# Turning the dictionary into a Pandas Dataframe
metadata_df = pd.DataFrame.from_dict(file_path_to_meta_data, orient="index")
# Resetting the index because currently the file path is the index 
metadata_df = metadata_df.reset_index()

In [12]:
metadata_df.head()

,index,File,Start Date,End Date,Subject,Experiment,Group,Box,Start Time,End Time,MSN
0,./data/med_pc_recording_files/2022-10-03_09h45...,C:\MED-PC\Data\2022-10-03_09h45m_Subject 1.1v1...,10/03/22,10/03/22,1.1v1.2,CD1_vs_C57_Comparison,Cage_1,1,09:45:25,10:46:28,C57_reward_competition


In [13]:
metadata_df.tail()

,index,File,Start Date,End Date,Subject,Experiment,Group,Box,Start Time,End Time,MSN
0,./data/med_pc_recording_files/2022-10-03_09h45...,C:\MED-PC\Data\2022-10-03_09h45m_Subject 1.1v1...,10/03/22,10/03/22,1.1v1.2,CD1_vs_C57_Comparison,Cage_1,1,09:45:25,10:46:28,C57_reward_competition


- Just getting the numbers out of the column that contains the cage information

## Inputting all the MED-PC log files

- Example of what the MED-PC Script looks like that was ran when recording the behaviors

In [14]:
with open("./data/MPC/C57_reward_competition.MPC") as f:
    lines = f.readlines()
    for line in lines[:100]:
        print(line)

\INPUTS



^port = 8





\OUTPUTS

^whitenoise = 1

^tone1 = 2

^tone2 = 3

^pump = 9

^fan = 16

^toneled = 17

^peled = 18







\ARRAYS



DIM P = 20000 \Port entry time stamp array



DIM Q = 250 \US delivery time stamp array (absolute)



DIM R = 250 \US time stamp array (relative to last CS)



DIM W = 2500 \ITI values used for CS



DIM S = 2500 \CS presentation values (absolute - every time yellow light turns on)



DIM N = 20000 \Port exit time stamp array



DIM K = 250 \CS type



DIM G = 250 \controlled_stimulus_seconds computer clock time (seconds on clock every time yellow light/tone turns on)



DIM H = 250 \controlled_stimulus_minutes computer clock time (minutes on clock every time yellow light/tone turns on)



DIM I = 250 \controlled_stimulus_hours computer clock time (hours every time yellow light/tone turns on)



DIM B = 2500 \port_entry_seconds computer clock time (seconds on clock every time red light turns on)



DIM F = 2500 \port_entry_minutes computer cloc

- We will be using the comments in the MED-PC script(which starts with the `\`) to create a name for the variables. By default, MED-PC uses a single letter as the name of the variable.
    - This will use the medpc2excel library found in https://github.com/cyf203/medpc2excel
- Example of the comments in the MED-PC script that we will use the names from:

```
\Variables

\A - Time since last CS

\B - Shock intensity

\C - Counter array

\D - Current ITI value

\E - CS ITI values for first few trials

\F - Shock duration

\G -

\H -

\I -

\J - Shock intensity repo

\K - CS type

\L -

\M - CS type repo

\N - Port exit time stamp array

\O -

\P - Port entry time stamp array

\Q - Sucrose delivery time stamp array (absolute)

\R - Sucrose delivery time stamp array (relative to last CS)

\S - CS presentation time stamp array

\T - Session timer

\U - Time since last CS presentation

\V - List of CS ITI values (tone + houselight)

\W - ITI values used for CS one each trial

\X -

\Y - Beam break monitor variable

\Z -
```

In [15]:
with open(all_med_pc_file[0]) as f:
    lines = f.readlines()
    for line in lines[:20]:
        print(line)

File: C:\MED-PC\Data\2022-10-03_09h45m_Subject 1.1v1.2.txt







Start Date: 10/03/22

End Date: 10/03/22

Subject: 1.1v1.2

Experiment: CD1_vs_C57_Comparison

Group: Cage_1

Box: 1

Start Time: 09:45:25

End Time: 10:46:28

MSN: C57_reward_competition

A:    5399.000

D:    8000.000

E:       0.000

L:       0.000

M:       0.000

O:       0.000

T:    1860.000



- **Please make sure that the corresponding `.mpc` file (aka the MED-PC script) that was ran to create the log file, is also in the same folder**

In [16]:
all_med_pc_file

['./data/med_pc_recording_files/2022-10-03_09h45m_Subject 1.1v1.2.txt']

In [17]:
concatted_medpc_df = extract.dataframe.get_medpc_dataframe_from_list_of_files(medpc_files=all_med_pc_file)
concatted_medpc_df = concatted_medpc_df.reset_index(drop=True)

In [18]:
concatted_medpc_df.head()

,(P)Portentry,(Q)USdelivery,(R)UStime,(W)ITIvalues,(S)CSpresentation,(N)Portexit,(K)CStype,(G)controlled_stimulus_secondscomputer,(H)controlled_stimulus_minutescomputer,(I)controlled_stimulus_hourscomputer,(B)port_entry_secondscomputer,(F)port_entry_minutescomputer,(J)port_entry_hourscomputer,date,subject,file_path
0,0.12,64.0,399.0,0.0,60.01,0.16,1.0,28.0,16.0,10.0,29.0,15.0,10.0,20221003,1.1v1.2,./data/med_pc_recording_files/2022-10-03_09h45...
1,10.41,144.0,399.0,0.0,140.01,10.43,1.0,49.0,17.0,10.0,39.0,15.0,10.0,20221003,1.1v1.2,./data/med_pc_recording_files/2022-10-03_09h45...
2,10.61,234.0,399.0,0.0,230.01,10.63,1.0,18.0,19.0,10.0,39.0,15.0,10.0,20221003,1.1v1.2,./data/med_pc_recording_files/2022-10-03_09h45...
3,11.20,314.0,399.0,0.0,310.01,11.42,1.0,38.0,20.0,10.0,39.0,15.0,10.0,20221003,1.1v1.2,./data/med_pc_recording_files/2022-10-03_09h45...
4,11.53,389.0,399.0,0.0,385.01,11.56,1.0,53.0,21.0,10.0,40.0,15.0,10.0,20221003,1.1v1.2,./data/med_pc_recording_files/2022-10-03_09h45...


In [19]:
concatted_medpc_df.tail()

,(P)Portentry,(Q)USdelivery,(R)UStime,(W)ITIvalues,(S)CSpresentation,(N)Portexit,(K)CStype,(G)controlled_stimulus_secondscomputer,(H)controlled_stimulus_minutescomputer,(I)controlled_stimulus_hourscomputer,(B)port_entry_secondscomputer,(F)port_entry_minutescomputer,(J)port_entry_hourscomputer,date,subject,file_path
5003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.0,NaN,NaN,20221003,1.1v1.2,./data/med_pc_recording_files/2022-10-03_09h45...
5004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,20221003,1.1v1.2,./data/med_pc_recording_files/2022-10-03_09h45...
5005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4385.0,NaN,NaN,20221003,1.1v1.2,./data/med_pc_recording_files/2022-10-03_09h45...
5006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.0,NaN,NaN,20221003,1.1v1.2,./data/med_pc_recording_files/2022-10-03_09h45...
5007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.0,NaN,NaN,20221003,1.1v1.2,./data/med_pc_recording_files/2022-10-03_09h45...


- Dropping all rows that don't have the CS presentation time

In [20]:
cs_time_df = concatted_medpc_df[concatted_medpc_df["(S)CSpresentation"] != 0.0].dropna(subset="(S)CSpresentation")

In [21]:
cs_time_df = cs_time_df[cs_time_df["(S)CSpresentation"] % 100 != 0]

In [22]:
cs_time_df["file_path"].unique()

array(['./data/med_pc_recording_files/2022-10-03_09h45m_Subject 1.1v1.2.txt'],
      dtype=object)

In [23]:
cs_time_df

,(P)Portentry,(Q)USdelivery,(R)UStime,(W)ITIvalues,(S)CSpresentation,(N)Portexit,(K)CStype,(G)controlled_stimulus_secondscomputer,(H)controlled_stimulus_minutescomputer,(I)controlled_stimulus_hourscomputer,(B)port_entry_secondscomputer,(F)port_entry_minutescomputer,(J)port_entry_hourscomputer,date,subject,file_path
0,0.12,64.0,399.0,0.0,60.01,0.16,1.0,28.0,16.0,10.0,29.0,15.0,10.0,20221003,1.1v1.2,./data/med_pc_recording_files/2022-10-03_09h45...
1,10.41,144.0,399.0,0.0,140.01,10.43,1.0,49.0,17.0,10.0,39.0,15.0,10.0,20221003,1.1v1.2,./data/med_pc_recording_files/2022-10-03_09h45...
2,10.61,234.0,399.0,0.0,230.01,10.63,1.0,18.0,19.0,10.0,39.0,15.0,10.0,20221003,1.1v1.2,./data/med_pc_recording_files/2022-10-03_09h45...
3,11.20,314.0,399.0,0.0,310.01,11.42,1.0,38.0,20.0,10.0,39.0,15.0,10.0,20221003,1.1v1.2,./data/med_pc_recording_files/2022-10-03_09h45...
4,11.53,389.0,399.0,0.0,385.01,11.56,1.0,53.0,21.0,10.0,40.0,15.0,10.0,20221003,1.1v1.2,./data/med_pc_recording_files/2022-10-03_09h45...
5,11.58,489.0,399.0,0.0,485.01,11.88,1.0,33.0,23.0,10.0,40.0,15.0,10.0,20221003,1.1v1.2,./data/med_pc_recording_files/2022-10-03_09h45...
6,11.90,584.0,399.0,0.0,580.01,12.16,1.0,8.0,25.0,10.0,40.0,15.0,10.0,20221003,1.1v1.2,./data/med_pc_recording_files/2022-10-03_09h45...
7,12.67,674.0,399.0,0.0,670.01,12.93,1.0,38.0,26.0,10.0,41.0,15.0,10.0,20221003,1.1v1.2,./data/med_pc_recording_files/2022-10-03_09h45...
8,14.85,754.0,399.0,0.0,750.01,14.91,1.0,58.0,27.0,10.0,43.0,15.0,10.0,20221003,1.1v1.2,./data/med_pc_recording_files/2022-10-03_09h45...
9,18.39,844.0,399.0,0.0,840.01,18.83,1.0,28.0,29.0,10.0,47.0,15.0,10.0,20221003,1.1v1.2,./data/med_pc_recording_files/2022-10-03_09h45...


In [24]:
cs_time_df["cs_minute_only"] = cs_time_df["(S)CSpresentation"] // 60
cs_time_df["cs_second_only"] = cs_time_df["(S)CSpresentation"] % 60


## Getting a list of the Tone presentation times

In [25]:
cs_presentation_time = cs_time_df[["(S)CSpresentation", "cs_minute_only", "cs_second_only"]]

In [26]:
cs_presentation_time

,(S)CSpresentation,cs_minute_only,cs_second_only
0,60.01,1.0,0.01
1,140.01,2.0,20.01
2,230.01,3.0,50.01
3,310.01,5.0,10.01
4,385.01,6.0,25.01
5,485.01,8.0,5.01
6,580.01,9.0,40.01
7,670.01,11.0,10.01
8,750.01,12.0,30.01
9,840.01,14.0,0.01


# Importing Video Data

In [27]:
from IPython.display import Video

1. Play the video, and go to time where the first tone is played in the dataframe above
    - Usually 1 minute
2. Note when the first tone is playing in the video
3. Subtract the difference between the time that the tone is playing in the video and in the dataframe
    - i.e. 1 minute 12 seconds - 1 minute = 12 seconds

In [28]:
all_video_paths = sorted(glob.glob("/nancy/user/riwata/projects/reward_competition_videos/Reward_Competition_D1_videos/*1*v*1*.mp4"))

In [29]:
all_video_paths = sorted(glob.glob("/nancy/user/riwata/projects/reward_competition_videos/Reward_Competition_D1_videos/*2*v*2*.mp4"))

In [30]:
all_video_paths

['/nancy/user/riwata/projects/reward_competition_videos/Reward_Competition_D1_videos/10-03-22_Test 11_6-1v6-2.mp4',
 '/nancy/user/riwata/projects/reward_competition_videos/Reward_Competition_D1_videos/10-03-22_Test 13_2-2v2-4.mp4',
 '/nancy/user/riwata/projects/reward_competition_videos/Reward_Competition_D1_videos/10-03-22_Test 14_2-1v2-3.mp4',
 '/nancy/user/riwata/projects/reward_competition_videos/Reward_Competition_D1_videos/10-03-22_Test 1_1-1v1-2.mp4',
 '/nancy/user/riwata/projects/reward_competition_videos/Reward_Competition_D1_videos/10-03-22_Test 27_2-2v2-3.mp4',
 '/nancy/user/riwata/projects/reward_competition_videos/Reward_Competition_D1_videos/10-03-22_Test 28_2-1v2-4.mp4',
 '/nancy/user/riwata/projects/reward_competition_videos/Reward_Competition_D1_videos/10-03-22_Test 3_2-1v2-2.mp4',
 '/nancy/user/riwata/projects/reward_competition_videos/Reward_Competition_D1_videos/10-03-22_Test 4_2-3v2-4.mp4',
 '/nancy/user/riwata/projects/reward_competition_videos/Reward_Competition_

In [31]:
video_path = all_video_paths[5]

# OVER HERE

In [32]:
video_file_name = "10-03-22_Test 10_5-3v5-4.mp4"

In [33]:
video_path = os.path.join("/nancy/user/riwata/projects/reward_competition_videos/Reward_Competition_D1_videos/", video_file_name)

In [34]:
video_path

'/nancy/user/riwata/projects/reward_competition_videos/Reward_Competition_D1_videos/10-03-22_Test 10_5-3v5-4.mp4'

## Calculating the predicted tone times in the video based on the difference

# NOTE: Change the value of the difference below

In [35]:
cs_time_difference = 11

In [36]:
cs_presentation_time["video_cs_time"] = cs_presentation_time["(S)CSpresentation"] + cs_time_difference

In [37]:
cs_presentation_time

,(S)CSpresentation,cs_minute_only,cs_second_only,video_cs_time
0,60.01,1.0,0.01,71.01
1,140.01,2.0,20.01,151.01
2,230.01,3.0,50.01,241.01
3,310.01,5.0,10.01,321.01
4,385.01,6.0,25.01,396.01
5,485.01,8.0,5.01,496.01
6,580.01,9.0,40.01,591.01
7,670.01,11.0,10.01,681.01
8,750.01,12.0,30.01,761.01
9,840.01,14.0,0.01,851.01


In [38]:
cs_presentation_time["video_cs_minute_only"] = cs_presentation_time["video_cs_time"] // 60
cs_presentation_time["video_cs_second_only"] = cs_presentation_time["video_cs_time"] % 60


# Trimming the video

In [39]:
video_object = VideoFileClip(video_path)

In [40]:
video_object.duration

1865.06

fixed func issue with: https://github.com/Zulko/moviepy/issues/1765


In [41]:
video_path

'/nancy/user/riwata/projects/reward_competition_videos/Reward_Competition_D1_videos/10-03-22_Test 10_5-3v5-4.mp4'

In [42]:
file_name = os.path.basename(video_path)
file_name_without_extention = os.path.splitext(file_name)[0]
file_name_without_extention = "_".join(file_name_without_extention.split())

In [43]:
file_name_without_extention

'10-03-22_Test_10_5-3v5-4'

In [44]:
clip_output_dir = "/nancy/user/riwata/projects/reward_competition_videos/pilot_3_day_1_cs_clips"

In [45]:
current_clip_output_dir = os.path.join(clip_output_dir, file_name_without_extention)
os.makedirs(current_clip_output_dir, exist_ok=True)

In [46]:
for index, row in cs_presentation_time.iterrows():
    cs_trimmed_video = video_object.subclip(row["video_cs_time"] - 5, row["video_cs_time"] + 15)
    cs_trimmed_file_name = "cs_number_{:02d}_date_{}.mp4".format(index + 1, file_name_without_extention)
    print(cs_trimmed_file_name)
    cs_trimmed_video.to_videofile(os.path.join(current_clip_output_dir, cs_trimmed_file_name))

cs_number_01_date_10-03-22_Test_10_5-3v5-4.mp4
Moviepy - Building video /nancy/user/riwata/projects/reward_competition_videos/pilot_3_day_1_cs_clips/10-03-22_Test_10_5-3v5-4/cs_number_01_date_10-03-22_Test_10_5-3v5-4.mp4.
Moviepy - Writing video /nancy/user/riwata/projects/reward_competition_videos/pilot_3_day_1_cs_clips/10-03-22_Test_10_5-3v5-4/cs_number_01_date_10-03-22_Test_10_5-3v5-4.mp4



Moviepy - Done !
Moviepy - video ready /nancy/user/riwata/projects/reward_competition_videos/pilot_3_day_1_cs_clips/10-03-22_Test_10_5-3v5-4/cs_number_01_date_10-03-22_Test_10_5-3v5-4.mp4
cs_number_02_date_10-03-22_Test_10_5-3v5-4.mp4
Moviepy - Building video /nancy/user/riwata/projects/reward_competition_videos/pilot_3_day_1_cs_clips/10-03-22_Test_10_5-3v5-4/cs_number_02_date_10-03-22_Test_10_5-3v5-4.mp4.
Moviepy - Writing video /nancy/user/riwata/projects/reward_competition_videos/pilot_3_day_1_cs_clips/10-03-22_Test_10_5-3v5-4/cs_number_02_date_10-03-22_Test_10_5-3v5-4.mp4



Moviepy - Done !
Moviepy - video ready /nancy/user/riwata/projects/reward_competition_videos/pilot_3_day_1_cs_clips/10-03-22_Test_10_5-3v5-4/cs_number_02_date_10-03-22_Test_10_5-3v5-4.mp4
cs_number_03_date_10-03-22_Test_10_5-3v5-4.mp4
Moviepy - Building video /nancy/user/riwata/projects/reward_competition_videos/pilot_3_day_1_cs_clips/10-03-22_Test_10_5-3v5-4/cs_number_03_date_10-03-22_Test_10_5-3v5-4.mp4.
Moviepy - Writing video /nancy/user/riwata/projects/reward_competition_videos/pilot_3_day_1_cs_clips/10-03-22_Test_10_5-3v5-4/cs_number_03_date_10-03-22_Test_10_5-3v5-4.mp4



Moviepy - Done !
Moviepy - video ready /nancy/user/riwata/projects/reward_competition_videos/pilot_3_day_1_cs_clips/10-03-22_Test_10_5-3v5-4/cs_number_03_date_10-03-22_Test_10_5-3v5-4.mp4
cs_number_04_date_10-03-22_Test_10_5-3v5-4.mp4
Moviepy - Building video /nancy/user/riwata/projects/reward_competition_videos/pilot_3_day_1_cs_clips/10-03-22_Test_10_5-3v5-4/cs_number_04_date_10-03-22_Test_10_5-3v5-4.mp4.
Moviepy - Writing video /nancy/user/riwata/projects/reward_competition_videos/pilot_3_day_1_cs_clips/10-03-22_Test_10_5-3v5-4/cs_number_04_date_10-03-22_Test_10_5-3v5-4.mp4



Moviepy - Done !
Moviepy - video ready /nancy/user/riwata/projects/reward_competition_videos/pilot_3_day_1_cs_clips/10-03-22_Test_10_5-3v5-4/cs_number_04_date_10-03-22_Test_10_5-3v5-4.mp4
cs_number_05_date_10-03-22_Test_10_5-3v5-4.mp4
Moviepy - Building video /nancy/user/riwata/projects/reward_competition_videos/pilot_3_day_1_cs_clips/10-03-22_Test_10_5-3v5-4/cs_number_05_date_10-03-22_Test_10_5-3v5-4.mp4.
Moviepy - Writing video /nancy/user/riwata/projects/reward_competition_videos/pilot_3_day_1_cs_clips/10-03-22_Test_10_5-3v5-4/cs_number_05_date_10-03-22_Test_10_5-3v5-4.mp4



...

Moviepy - Done !
Moviepy - video ready /nancy/user/riwata/projects/reward_competition_videos/pilot_3_day_1_cs_clips/10-03-22_Test_10_5-3v5-4/cs_number_15_date_10-03-22_Test_10_5-3v5-4.mp4
cs_number_16_date_10-03-22_Test_10_5-3v5-4.mp4
Moviepy - Building video /nancy/user/riwata/projects/reward_competition_videos/pilot_3_day_1_cs_clips/10-03-22_Test_10_5-3v5-4/cs_number_16_date_10-03-22_Test_10_5-3v5-4.mp4.
Moviepy - Writing video /nancy/user/riwata/projects/reward_competition_videos/pilot_3_day_1_cs_clips/10-03-22_Test_10_5-3v5-4/cs_number_16_date_10-03-22_Test_10_5-3v5-4.mp4



Moviepy - Done !
Moviepy - video ready /nancy/user/riwata/projects/reward_competition_videos/pilot_3_day_1_cs_clips/10-03-22_Test_10_5-3v5-4/cs_number_16_date_10-03-22_Test_10_5-3v5-4.mp4
cs_number_17_date_10-03-22_Test_10_5-3v5-4.mp4
Moviepy - Building video /nancy/user/riwata/projects/reward_competition_videos/pilot_3_day_1_cs_clips/10-03-22_Test_10_5-3v5-4/cs_number_17_date_10-03-22_Test_10_5-3v5-4.mp4.
Moviepy - Writing video /nancy/user/riwata/projects/reward_competition_videos/pilot_3_day_1_cs_clips/10-03-22_Test_10_5-3v5-4/cs_number_17_date_10-03-22_Test_10_5-3v5-4.mp4



Moviepy - Done !
Moviepy - video ready /nancy/user/riwata/projects/reward_competition_videos/pilot_3_day_1_cs_clips/10-03-22_Test_10_5-3v5-4/cs_number_17_date_10-03-22_Test_10_5-3v5-4.mp4
cs_number_18_date_10-03-22_Test_10_5-3v5-4.mp4
Moviepy - Building video /nancy/user/riwata/projects/reward_competition_videos/pilot_3_day_1_cs_clips/10-03-22_Test_10_5-3v5-4/cs_number_18_date_10-03-22_Test_10_5-3v5-4.mp4.
Moviepy - Writing video /nancy/user/riwata/projects/reward_competition_videos/pilot_3_day_1_cs_clips/10-03-22_Test_10_5-3v5-4/cs_number_18_date_10-03-22_Test_10_5-3v5-4.mp4



Moviepy - Done !
Moviepy - video ready /nancy/user/riwata/projects/reward_competition_videos/pilot_3_day_1_cs_clips/10-03-22_Test_10_5-3v5-4/cs_number_18_date_10-03-22_Test_10_5-3v5-4.mp4
cs_number_19_date_10-03-22_Test_10_5-3v5-4.mp4
Moviepy - Building video /nancy/user/riwata/projects/reward_competition_videos/pilot_3_day_1_cs_clips/10-03-22_Test_10_5-3v5-4/cs_number_19_date_10-03-22_Test_10_5-3v5-4.mp4.
Moviepy - Writing video /nancy/user/riwata/projects/reward_competition_videos/pilot_3_day_1_cs_clips/10-03-22_Test_10_5-3v5-4/cs_number_19_date_10-03-22_Test_10_5-3v5-4.mp4



Moviepy - Done !
Moviepy - video ready /nancy/user/riwata/projects/reward_competition_videos/pilot_3_day_1_cs_clips/10-03-22_Test_10_5-3v5-4/cs_number_19_date_10-03-22_Test_10_5-3v5-4.mp4


In [47]:
cs_trimmed_times_name = "cs_times_{}_.csv".format(file_name_without_extention)


In [48]:
cs_presentation_time.to_csv(os.path.join(current_clip_output_dir, cs_trimmed_times_name))